In [8]:
import numpy as np
import tensorflow as tf
x = np.arange(0, 10)
# create dataset object from numpy array
dx = tf.data.Dataset.from_tensor_slices((x,x))
# create a one-shot iterator
iterator = dx.make_one_shot_iterator()
# extract an element
next_element = iterator.get_next()

In [24]:
dataset = tf.data.Dataset.from_tensor_slices((np.array([1.0, 2.0, 3.0, 4.0, 5.0]),np.zeros((5,3))))
iterator = dataset.make_one_shot_iterator()
one_element = iterator.get_next()
with tf.Session() as sess:
    for i in range(5):
        print(sess.run(one_element))
        

(1.0, array([0., 0., 0.]))
(2.0, array([0., 0., 0.]))
(3.0, array([0., 0., 0.]))
(4.0, array([0., 0., 0.]))
(5.0, array([0., 0., 0.]))


In [102]:
import numpy as np
import os
import tensorflow as tf
from tensorflow.contrib.framework import arg_scope
from tensorflow.contrib.layers import fully_connected, batch_norm
from tensorflow.examples.tutorials.mnist import input_data

###### Do not modify here ###### 
def fully_connected_layer(x_inputs, out_dim, name='fc'):
    """ Low level method
        x_inputs: a batch examples [batch_size, feature_dims]
        out_dim: neurons in this layer.
    """ 
    in_dim = x_inputs.shape[-1] # feature_dims
    
    with tf.variable_scope(name, reuse=tf.AUTO_REUSE):
        weights = tf.get_variable("weights", shape=[in_dim, out_dim])
        bias = tf.get_variable("bias", shape=[out_dim])
        out = tf.matmul(x_inputs, weights) + bias
        return out
# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

reset_graph()


# mnist = input_data.read_data_sets("data", one_hot=True)
# mnist = input_data.read_data_sets("data")
if not mnist:
    mnist = input_data.read_data_sets("data")


# training on MNIST but only on digits 0 to 4
X_train1 = mnist.train.images
y_train1 = mnist.train.labels

X_test1 = mnist.test.images
y_test1 = mnist.test.labels

###### Do not modify here ###### 

#get next batch in order
def next_batch(batch_size,iteration, data, labels):
    start = batch_size*iteration
    end = batch_size*(iteration+1)
    return data[start:end], labels[start:end]

#const parameters
n_inputs = 784  # MNIST
n_outputs = 10

#adjustable parameters
N_neurons = 128
learning_rate = 0.01
momentum = 0.25
epochs = 2
batch_size = 128  #for GPU optimize
dropout = 0.5

train_iterator = tf.data.Dataset.from_tensor_slices((mnist.train.images,mnist.train.labels)).make_one_shot_iterator()




# X,y = train_iterator.get_next()
# X = tf.manip.reshape(X,(784,1))
# y = tf.manip.reshape(y,(1,))

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")

he_init = tf.contrib.layers.variance_scaling_initializer()

W1 = tf.sigmoid(fully_connected_layer(X,N_neurons,name='L1'))
W2 = tf.sigmoid(fully_connected_layer(W1,N_neurons,name='L2'))
y_hat = fully_connected_layer(W2, n_outputs,name = 'L3')


cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=y_hat)
loss = tf.reduce_mean(cross_entropy, name="loss")

#use AdamOptimizer
optimizer = tf.train.AdamOptimizer(learning_rate, momentum)
training_op = optimizer.minimize(loss,name = "training_op")

correct = tf.nn.in_top_k(y_hat,y,1)
accuracy = tf.reduce_mean(tf.cast(correct,tf.float32), name="accuracy")


init = tf.global_variables_initializer()

with tf.Session() as sess:
    init.run()
    
    for epoch in range(epochs):
        for iteration in range(len(X_train1)//batch_size):
            X_batch, y_batch = next_batch(batch_size,iteration,X_train1, y_train1)
            sess.run(training_op, feed_dict={ X: X_batch, y: y_batch})
#             sess.run(training_op )
            
        acc_train = accuracy.eval(feed_dict={ X: X_batch, y: y_batch})
        
    acc_test = accuracy.eval(feed_dict={X:  X_test1, y:  y_test1})
    
    print( "Test accuracy:", acc_test)

Test accuracy: 0.966


In [51]:
iterator = tf.data.Iterator.from_structure(tf.int64, tf.TensorShape([]))

dataset_range = tf.data.Dataset.range(10)
range_initializer = iterator.make_initializer(dataset_range)

dataset_evens = dataset_range.filter(lambda x: x % 2 == 0)

evens_initializer = iterator.make_initializer(dataset_evens)
next_element = iterator.get_next()
print(next_element)

Tensor("IteratorGetNext_1:0", shape=(), dtype=int64)
